In [3]:
import os
import sys
import datetime
import numpy as np
import pandas as pd
import yfinance as yf

import matplotlib.pyplot as plt
%matplotlib inline

Data Exploration - To get specific ticker data

In [11]:
start_date = datetime.datetime(2015, 1, 1).date()
end_date = datetime.datetime.now().date()
start_date, end_date

(datetime.date(2015, 1, 1), datetime.date(2025, 11, 1))

In [12]:
tickers = "NVDA"

In [13]:
nvda = yf.Ticker(tickers)

In [14]:
historical_data = nvda.history(start = start_date, end = end_date, interval = '1d')

In [24]:
historical_data.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2015-01-02 00:00:00-05:00,0.483038,0.486638,0.475360,0.483038,113680000,0.0,0.0
2015-01-05 00:00:00-05:00,0.483038,0.484478,0.472720,0.474880,197952000,0.0,0.0
2015-01-06 00:00:00-05:00,0.475600,0.476079,0.460002,0.460482,197764000,0.0,0.0
2015-01-07 00:00:00-05:00,0.463841,0.467921,0.457843,0.459282,321808000,0.0,0.0
2015-01-08 00:00:00-05:00,0.464561,0.479439,0.464322,0.476559,283780000,0.0,0.0


In [25]:
historical_data.describe()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
count,2725.000000,2725.000000,2725.000000,2725.000000,2.725000e+03,2725.000000,2725.000000
mean,30.522505,31.047179,29.947724,30.527171,4.491592e+08,0.000072,0.005138
std,46.592921,47.318086,45.759778,46.579813,2.534340e+08,0.000633,0.206296
min,0.463602,0.467921,0.454483,0.459282,5.244800e+07,0.000000,0.000000
25%,3.743801,3.802065,3.655536,3.729916,2.848855e+08,0.000000,0.000000
50%,8.788758,9.008670,8.674169,8.775578,3.983550e+08,0.000000,0.000000
75%,27.629575,28.091148,27.104983,27.645567,5.435010e+08,0.000000,0.000000
max,207.979996,212.190002,204.779999,207.039993,3.692928e+09,0.010000,10.000000


In [26]:
fig = plt.figure()

plt.plot(historical_data.Close)

plt.legend(["Close", "Open"])

Feature Engineering - Dataset